# Weather Dataset Preprocessing and feature Engineering

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@48d1da41
session = org.apache.spark.sql.SparkSession@48d1da41
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

In [3]:
weatherDF.count()

5669039

In [4]:
weatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string (

## SkyConditionFeatures

In [8]:
var addedFeatureColumns = Seq(
"feature_most_critical_sky",
"feature_num_cloud_layers",
"feature_cloud_risk_score",
"feature_has_overcast",
"feature_has_broken",
"feature_has_obscured",
"feature_is_clear",
"feature_lowest_cloud_height",
"feature_ceiling",
"feature_has_low_ceiling",
"feature_visibility_category",
)

var df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0----------------------------
 feature_most_critical_sky   | CLR   
 feature_num_cloud_layers    | 1     
 feature_cloud_risk_score    | 0.0   
 feature_has_overcast        | false 
 feature_has_broken          | false 
 feature_has_obscured        | false 
 feature_is_clear            | true  
 feature_lowest_cloud_height | 99999 
 feature_ceiling             | 99999 
 feature_has_low_ceiling     | 0     
 feature_visibility_category | IFR   
only showing top 1 row



addedFeatureColumns = List(feature_most_critical_sky, feature_num_cloud_layers, feature_cloud_risk_score, feature_has_overcast, feature_has_broken, feature_has_obscured, feature_is_clear, feature_lowest_cloud_height, feature_ceiling, feature_has_low_ceiling, feature_visibility_category)
df = [feature_most_critical_sky: string, feature_num_cloud_layers: int ... 9 more fields]


[feature_most_critical_sky: string, feature_num_cloud_layers: int ... 9 more fields]

## VisibilityFeatures

In [19]:
addedFeatureColumns = Seq(
"feature_visibility_miles",
"feature_visibility_km",
"feature_visibility_category",
"feature_visibility_risk_score",
"feature_is_low_visibility",
"feature_is_very_low_visibility",
"feature_visibility_normalized",
"feature_visibility_inverse"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0-------------------------------
 feature_visibility_miles       | 1.0   
 feature_visibility_km          | 1.609 
 feature_visibility_category    | IFR   
 feature_visibility_risk_score  | 3.0   
 feature_is_low_visibility      | 1     
 feature_is_very_low_visibility | 0     
 feature_visibility_normalized  | 0.1   
 feature_visibility_inverse     | 1.0   
only showing top 1 row



lastException = null
addedFeatureColumns = List(feature_visibility_miles, feature_visibility_km, feature_visibility_category, feature_visibility_risk_score, feature_is_low_visibility, feature_is_very_low_visibility, feature_visibility_normalized, feature_visibility_inverse)
df = [feature_visibility_miles: double, feature_visibility_km: double ... 6 more fields]


[feature_visibility_miles: double, feature_visibility_km: double ... 6 more fields]

In [20]:
// Obtenir les valeurs distinctes
var distinctValues = df.select("feature_visibility_category")
  .distinct()
  .orderBy("feature_visibility_category")

// Afficher les résultats
distinctValues.show()

+---------------------------+
|feature_visibility_category|
+---------------------------+
|                        IFR|
|                    IFR_LOW|
|                       LIFR|
|                       MVFR|
|                        VFR|
|                   VFR_HIGH|
+---------------------------+



distinctValues = [feature_visibility_category: string]


[feature_visibility_category: string]

## WeatherInteractionFeatures

In [15]:
addedFeatureColumns = Seq(
"feature_weather_severity_index",
"feature_is_vfr_conditions",
"feature_is_ifr_conditions",
"feature_requires_cat_ii",
"feature_operations_risk_level"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------------------------------
 feature_weather_severity_index | 1.7999999999999998 
 feature_is_vfr_conditions      | 0                  
 feature_is_ifr_conditions      | 1                  
 feature_requires_cat_ii        | 0                  
 feature_operations_risk_level  | 2                  
only showing top 1 row



addedFeatureColumns = List(feature_weather_severity_index, feature_is_vfr_conditions, feature_is_ifr_conditions, feature_requires_cat_ii, feature_operations_risk_level)
df = [feature_weather_severity_index: double, feature_is_vfr_conditions: int ... 3 more fields]


[feature_weather_severity_index: double, feature_is_vfr_conditions: int ... 3 more fields]

## WeatherTypeFeatureGenerator

In [21]:
addedFeatureColumns = Seq(
"intensity_heavy",
"intensity_light",
"weather_intensity",
"has_thunderstorm",
"has_freezing_precip",
"has_freezing",
"has_precipitation",
"has_obscuration",
"has_visible_moisture",
"has_hazardous",
"has_rain",
"has_snow",
"has_hail",
"extracted_codes",
"weather_hazard_level",
"Icing_Risk_Flag",
"Icing_Risk_Level"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0------------------------
 intensity_heavy      | 0        
 intensity_light      | 0        
 weather_intensity    | moderate 
 has_thunderstorm     | 0        
 has_freezing_precip  | 0        
 has_freezing         | 0        
 has_precipitation    | 0        
 has_obscuration      | 0        
 has_visible_moisture | 0        
 has_hazardous        | 0        
 has_rain             | 0        
 has_snow             | 0        
 has_hail             | 0        
 extracted_codes      |          
 weather_hazard_level | 0        
 Icing_Risk_Flag      | 0        
 Icing_Risk_Level     | 0        
only showing top 1 row



addedFeatureColumns = List(intensity_heavy, intensity_light, weather_intensity, has_thunderstorm, has_freezing_precip, has_freezing, has_precipitation, has_obscuration, has_visible_moisture, has_hazardous, has_rain, has_snow, has_hail, extracted_codes, weather_hazard_level, Icing_Risk_Flag, Icing_Risk_Level)
df = [intensity_heavy: int, intensity_light: int ... 15 more fields]


[intensity_heavy: int, intensity_light: int ... 15 more fields]

In [22]:
// Obtenir les valeurs distinctes
val distinctValues = df.select("weather_intensity")
  .distinct()
  .orderBy("weather_intensity")

// Afficher les résultats
distinctValues.show()

+-----------------+
|weather_intensity|
+-----------------+
|            heavy|
|            light|
|         moderate|
+-----------------+



distinctValues = [weather_intensity: string]


[weather_intensity: string]

In [25]:
import org.apache.spark.sql.functions.col

// Filtrer sur weather_intensity == "heavy"
val heavyWeatherDF = df.filter(col("weather_intensity") === "heavy")

// Afficher le résultat
heavyWeatherDF.show(1, 10000, true)

-RECORD 0---------------------
 intensity_heavy      | 1     
 intensity_light      | 0     
 weather_intensity    | heavy 
 has_thunderstorm     | 0     
 has_freezing_precip  | 0     
 has_freezing         | 0     
 has_precipitation    | 1     
 has_obscuration      | 1     
 has_visible_moisture | 1     
 has_hazardous        | 0     
 has_rain             | 1     
 has_snow             | 0     
 has_hail             | 0     
 extracted_codes      | RA BR 
 weather_hazard_level | 2     
 Icing_Risk_Flag      | 0     
 Icing_Risk_Level     | 0     
only showing top 1 row



heavyWeatherDF = [intensity_heavy: int, intensity_light: int ... 15 more fields]


[intensity_heavy: int, intensity_light: int ... 15 more fields]

## Original DF

In [29]:
var featureColumns = Seq(
"DryBulbCelsius",
"DryBulbCelsiusFlag",
"DryBulbFarenheit",
"DryBulbFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------
 DryBulbCelsius       | -1.0 
 DryBulbCelsiusFlag   |      
 DryBulbFarenheit     | 30.0 
 DryBulbFarenheitFlag |      
only showing top 1 row



featureColumns = List(DryBulbCelsius, DryBulbCelsiusFlag, DryBulbFarenheit, DryBulbFarenheitFlag)
df = [DryBulbCelsius: double, DryBulbCelsiusFlag: string ... 2 more fields]


[DryBulbCelsius: double, DryBulbCelsiusFlag: string ... 2 more fields]

In [30]:
featureColumns = Seq(
"DewPointCelsius",
"DewPointCelsiusFlag",
"DewPointFarenheit",
"DewPointFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

featureColumns = List(DewPointCelsius, DewPointCelsiusFlag, DewPointFarenheit, DewPointFarenheitFlag)
df = [DewPointCelsius: double, DewPointCelsiusFlag: string ... 2 more fields]


-RECORD 0----------------------
 DewPointCelsius       | -12.0 
 DewPointCelsiusFlag   |       
 DewPointFarenheit     | 10.0  
 DewPointFarenheitFlag |       
only showing top 1 row



[DewPointCelsius: double, DewPointCelsiusFlag: string ... 2 more fields]

In [31]:
featureColumns = Seq(
"WetBulbCelsius",
"WetBulbCelsiusFlag",
"WetBulbFarenheit",
"WetBulbFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------
 WetBulbCelsius       | -4.9 
 WetBulbCelsiusFlag   |      
 WetBulbFarenheit     | 23.0 
 WetBulbFarenheitFlag |      
only showing top 1 row



featureColumns = List(WetBulbCelsius, WetBulbCelsiusFlag, WetBulbFarenheit, WetBulbFarenheitFlag)
df = [WetBulbCelsius: double, WetBulbCelsiusFlag: string ... 2 more fields]


[WetBulbCelsius: double, WetBulbCelsiusFlag: string ... 2 more fields]

In [32]:
featureColumns = Seq(
"RelativeHumidity",
"RelativeHumidityFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------
 RelativeHumidity     | 43.0 
 RelativeHumidityFlag |      
only showing top 1 row



featureColumns = List(RelativeHumidity, RelativeHumidityFlag)
df = [RelativeHumidity: double, RelativeHumidityFlag: string]


[RelativeHumidity: double, RelativeHumidityFlag: string]

In [33]:
featureColumns = Seq(
"HourlyPrecip",
"HourlyPrecipFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

featureColumns = List(HourlyPrecip, HourlyPrecipFlag)
df = [HourlyPrecip: string, HourlyPrecipFlag: string]


-RECORD 0---------------
 HourlyPrecip     |     
 HourlyPrecipFlag |     
only showing top 1 row



[HourlyPrecip: string, HourlyPrecipFlag: string]

In [34]:
featureColumns = Seq(
"WindSpeed",
"WindSpeedFlag",
"WindDirection",
"WindDirectionFlag",
"ValueForWindCharacter",
"ValueForWindCharacterFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------------
 WindSpeed                 | 5.0   
 WindSpeedFlag             |       
 WindDirection             | 120.0 
 WindDirectionFlag         |       
 ValueForWindCharacter     |       
 ValueForWindCharacterFlag |       
only showing top 1 row



featureColumns = List(WindSpeed, WindSpeedFlag, WindDirection, WindDirectionFlag, ValueForWindCharacter, ValueForWindCharacterFlag)
df = [WindSpeed: double, WindSpeedFlag: string ... 4 more fields]


[WindSpeed: double, WindSpeedFlag: string ... 4 more fields]

In [35]:
featureColumns = Seq(
"Visibility",
"VisibilityFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------
 Visibility     | 10.0 
 VisibilityFlag |      
only showing top 1 row



featureColumns = List(Visibility, VisibilityFlag)
df = [Visibility: double, VisibilityFlag: string]


[Visibility: double, VisibilityFlag: string]